In [1]:
#pip install -U google-genai
#%pip install pandas tqdm


In [ ]:
#export GEMINI_API_KEY="###"


In [3]:
import os
import time
from datetime import datetime

import pandas as pd
from tqdm import tqdm
from google import genai
from google.genai import types

/Users/mengyingwang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# 如果没有在系统里设置 GEMINI_API_KEY，可以取消下一行注释直接写入
os.environ["GEMINI_API_KEY"] = "###"

In [5]:
MODEL_NAME = "gemini-2.5-flash"
RATE_SLEEP = 1.0   # 每次调用后的短暂停顿，避免触发限制

In [7]:
# 5 个 prompt 模板
PROMPTS = [
    lambda x: f'Rewrite the following paragraph:\n1: Paragraph: "{x}"\nRewritten version:',
    lambda x: f'How would you rephrase this paragraph while preserving its original meaning?\nParagraph: "{x}"\nRephrased version:',
    lambda x: f'Rephrase the following paragraph without changing the main content:\nParagraph: "{x}"\nRephrased version:',
    lambda x: (
        "Rephrase the following paragraph while preserving its meaning. Follow these steps:\n"
        "1️⃣ Split the paragraph into individual sentences.\n"
        "2️⃣ Rephrase each sentence naturally while keeping the overall flow.\n"
        "3️⃣ Combine the rephrased sentences into a coherent paragraph.\n\n"
        f'Paragraph: "{x}"\nRephrased version:'
    ),
    lambda x: (
        "Imagine you are an advanced language model capable of rephrasing text while preserving its original meaning. "
        "If this were your paragraph, how would you naturally rephrase it?\n\n"
        f'Paragraph: "{x}"\nYour rephrased version:'
    ),
]

In [8]:
# 初始化客户端
client = genai.Client()

In [ ]:
def call_gemini(prompt_text):

    resp = client.models.generate_content(
        model=MODEL_NAME,
        contents=prompt_text
        #config=config
    )
    #print(resp)
    return (resp.text)

In [ ]:
# 读取数据（需要有一列名为 'text'）
df = pd.read_csv("###/original text.csv")
df = df.iloc[380:410]


In [11]:
# 结果容器
p_cols = {f"p{i}": [] for i in range(1, 6)}

In [12]:
# 主循环
for raw_text in tqdm(df["paragraph"], desc="Rephrasing with 5 prompts"):
    text = (str(raw_text) or "").strip()
    for i, tmpl in enumerate(PROMPTS, start=1):
        prompt = tmpl(text)
        #print(prompt)
        out = call_gemini(prompt)
        p_cols[f"p{i}"].append(out)
        time.sleep(RATE_SLEEP)

Rephrasing with 5 prompts: 100%|██████████| 30/30 [27:25<00:00, 54.84s/it]


In [13]:
# 合并结果并保存
out_df = df.copy()
for k, v in p_cols.items():
    out_df[k] = v

In [ ]:
out_df.to_csv('###/Gemini.csv', index=False)
print("Done!")


Done!
